# Data Cleaning

This notebook will clean the raw releases/disposals/disposals-transfer CSV files and will save them for analysis.

In [ ]:
import sys
from pathlib import Path

# Add project root manually
project_root = Path.cwd().parents[0]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd

from src.config import *
from src.file_util import *

In [ ]:
ensure_directory_exists(PROCESSED_DIR)


## Load Geolocation Data

In [ ]:
geo_raw_file_path = str(RAW_DIR) + '/' + get_filename_from_url(DATA_URLS['geolocation'])
geo_raw_df = pd.read_csv(geo_raw_file_path, encoding='ISO-8859-1', low_memory=False)

geo_renames = {
    'Year of last filed report / Année de déclaration la plus récente': 'last_report_year',
    'NPRI ID / ID INRP': 'npri_id',
    'Company Name / Raison Sociale': 'company_name',
    'Facility Name / Nom de l\'installation': 'facility_name',
    'Address line 1 / Première ligne dadresse': 'address_line_1',
    'Address line 2 / Deuxième ligne dadresse': 'address_line_2',
    'City / Ville': 'city',
    'Province / Province': 'province',
    'Postal Code / Code postal': 'postal_code',
    'Physical Land Survey Description / Description de l\'arpentage': 'land_survey_description',
    'National Topographical Description / Description topographique nationale': 'topographical_description',
    'Latitude / Latitude': 'latitude',
    'Longitude / Longitude': 'longitude',
    'Datum / Datum': 'datum',
    'NAICS / Code SCIAN': 'naics_code',
    'Key Industrial Sector Code / Code des Secteurs industriels clés': 'key_industrial_sector_code',
    'Key Industrial Sector (English) / Secteurs industriels clés (Anglais)': 'key_industrial_sector_en',
    'Key Industrial Sector (French) / Secteurs industriels clés (Français)': 'key_industrial_sector_fr',
    'Census Division (CD) Unique ID / No unique de la Division de recensement (DR)': 'census_division_id',
    'Census Division (CD) Name / Nom de la Division de recensement (DR)': 'census_division_name',
    'Census Sub Division (CSD) Unique ID / No unique de la Subdivision de recensement (SDR)': 'census_subdivision_id',
    'Census Sub Division (CSD) Name / Nom de la Subdivision de recensement (SDR)': 'census_subdivision_name',
    'Census Metropolitan Area (CMA) Unique ID / No unique de la Région métropolitaine de recensement (RMR)': 'cma_id',
    'Census Metropolitan Area (CMA) Name / Nom de la Région métropolitaine de recensement (RMR)': 'cma_name',
    'Economic Region (ER) Unique ID / No unique de la Région économique (RE)': 'economic_region_id',
    'Economic Region (ER) Name / Nom de la Région économique (RE)': 'economic_region_name',
    'Forward Sortation Area (FSA) / Région de tri d\'acheminement (RTA)': 'fsa',
    'Ecozone Unique ID / No unique de lécozone': 'ecozone_id',
    'English Ecozone Name / Nom anglais de lécozone': 'ecozone_name_en',
    'French Ecozone Name / Nom français de lécozone': 'ecozone_name_fr',
    'Unique ID of the Major Drainage Area from the Water Survey of Canada (WSC) / No unique de laire de drainage principale des Relevés hydrologiques du Canada (RHC)': 'major_drainage_area_id',
    'Major Drainage Area English Name / Nom anglais de laire de drainage principale': 'major_drainage_area_name_en',
    'Major Drainage Area French Name / Nom français de laire de drainage principale': 'major_drainage_area_name_fr',
    'Sulphur Oxide Management Area (SOMA) / Zone de gestion des oxydes de soufre (ZGOS)': 'soma_zone',
    'Ontario Pollution Emission Management Area (PEMA) / Zone de gestion des émission de polluants (ZGEP) de l\'Ontario': 'ontario_pema_zone',
    'Quebec Pollution Emission Management Area (PEMA) / Zone de gestion des émission de polluants (ZGEP) du Québec': 'quebec_pema_zone',
    'Quebec City-Windsor Corridor / Corridor Québec-Windsor': 'qc_windsor_corridor',
}

geo_raw_df.rename(columns=geo_renames, inplace=True)


# Convert to numeric and coerce invalid entries (e.g., empty strings) to NaN
geo_raw_df['Latitude'] = pd.to_numeric(geo_raw_df['latitude'], errors='coerce')
geo_raw_df['Longitude'] = pd.to_numeric(geo_raw_df['longitude'], errors='coerce')

# Drop rows with NaN or non-finite values
geo_df_clean = geo_raw_df[np.isfinite(geo_raw_df['latitude']) & np.isfinite(geo_raw_df['longitude'])].copy()

print('before dropping NaN: ', len(geo_raw_df), ', after: ', len(geo_df_clean))

geo_df_clean.to_csv(str(PROCESSED_DIR) + '/geolocation.csv', index=False)

display(geo_df_clean.head(3))
display(geo_df_clean.tail(3))

## Loading Releases Data

In [ ]:
releases_raw_file_path = str(RAW_DIR) + '/' + get_filename_from_url(DATA_URLS['releases'])
releases_df = pd.read_csv(releases_raw_file_path, encoding='ISO-8859-1', low_memory=False)

column_renames = {
    'Reporting_Year / Année': 'reporting_year',
    'NPRI_ID / No_INRP': 'npri_id',
    "Company_Name / Dénomination_sociale_de_l'entreprise": 'company_name',
    'Facility_Name / Installation': 'facility_name',
    'NAICS / Code_SCIAN': 'naics_code',
    'NAICS Title EN / Titre Code SCIAN EN': 'naics_title_en',
    'NAICS Title FR / Titre Code SCIAN FR': 'naics_title_fr',
    'PROVINCE': 'province',
    'CAS_Number / No_CAS': 'cas_number',
    'Substance Name (English) / Nom de substance (Anglais)': 'substance_name_en',
    'Substance Name (French) / Nom de substance (Français)': 'substance_name_fr',
    'Group (English) / Groupe (Anglais)': 'group_en',
    'Group (French) / Groupe (Français)': 'group_fr',
    'Category (English) / Catégorie (Anglais)': 'category_en',
    'Category (French) / Catégorie (Français)': 'category_fr',
    'Quantity / Quantité': 'quantity',
    'Units / Unités': 'units',
    'Estimation_Method / Méthode_destimation': 'estimation_method',
}

releases_df.rename(columns=column_renames, inplace=True)

releases_df.to_csv(str(PROCESSED_DIR) + '/releases.csv', index=False)

display(releases_df.head(3))
display(releases_df.tail(3))


## Loading Disposals Data

In [ ]:
disposals_raw_file_path = str(RAW_DIR) + '/' + get_filename_from_url(DATA_URLS['disposals'])
disposals_df = pd.read_csv(disposals_raw_file_path, encoding='ISO-8859-1', low_memory=False)

disposals_df.rename(columns=column_renames, inplace=True)

disposals_df.to_csv(str(PROCESSED_DIR) + '/disposals.csv', index=False)

display(disposals_df.head(3))
display(disposals_df.tail(3))


## Loading Transfers Data

In [ ]:
transfers_file_path = str(RAW_DIR) + '/' + get_filename_from_url(DATA_URLS['transfers'])
transfers_df = pd.read_csv(transfers_file_path, encoding='ISO-8859-1', low_memory=False)

transfers_df.rename(columns=column_renames, inplace=True)

transfers_df.to_csv(str(PROCESSED_DIR) + '/transfers.csv', index=False)

display(transfers_df.head(3))
display(transfers_df.tail(3))
